In [1]:
%load_ext autoreload
%autoreload 2

from corvus_python.pyspark.storage import LocalFileSystemStorageConfiguration
from corvus_python.pyspark.utilities import create_spark_session

file_system_configuration = LocalFileSystemStorageConfiguration("sample_entrypoints/data/")

In [2]:
spark = create_spark_session("test_notebook", file_system_configuration, enable_hive_support=False)

:: loading settings :: url = jar:file:/workspaces/Corvus.Python/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
com.microsoft.sqlserver#mssql-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3dcd9040-cb35-4695-b836-12349902bb6e;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found com.microsoft.sqlserver#mssql-jdbc;12.6.0.jre11 in central
:: resolution report :: resolve 259ms :: artifacts dl 14ms
	:: modules in use:
	com.microsoft.sqlserver#mssql-jdbc;12.6.0.jre11 from central in [default]
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   

24/02/15 13:20:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [18]:
import pandas as pd

data = {
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Metric1': [10, 20, 30],
    'Metric2': [40, 50, 60],
    'Metric3': [70, 80, 90]
}

df = pd.DataFrame(data)

df = spark.createDataFrame(df)

df.show()

/workspaces/Corvus.Python/.venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/workspaces/Corvus.Python/.venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-------+-------+-------+-------+
|   Name|Metric1|Metric2|Metric3|
+-------+-------+-------+-------+
|  Alice|     10|     40|     70|
|    Bob|     20|     50|     80|
|Charlie|     30|     60|     90|
+-------+-------+-------+-------+



In [40]:
from dataclasses import dataclass
from pyspark.sql import DataFrame
from typing import List

@dataclass
class RenameMapping:
    original_name: str
    new_name: str


# Replace with pyspark.sql.DataFrame.unpivot when using version 3.4.0. Function signature is the same.
def spark_unpivot(df, ids, values=None, variableColumnName='variable', valueColumnName='value'):
    if not values:
        values = [c for c in df.columns if c not in ids]
    
    values_expr = ', '.join([f"'{c}', {c}" for c in values])

    stack_select_expr = f"stack({len(values)}, {values_expr}) as ({variableColumnName}, {valueColumnName})"
    print(stack_select_expr)

    return df.selectExpr(*ids, stack_select_expr)

def spark_unpivot_with_rename(
        df: DataFrame,
        ids: List[str],
        values: List[str] = None,
        variableColumnName: str = 'variable',
        valueColumnName: str = 'value',
        renameMappings: List[RenameMapping] = None
        ):
    if not values:
        values = [c for c in df.columns if c not in ids]
    
    if renameMappings:
        value_pairs = []
        for v in values:
            rm = next((rm for rm in renameMappings if rm.original_name == v), None)
            if rm:
                value_pair = f"'{rm.new_name}', {rm.original_name}"
            else:
                value_pair = f"'{v}', {v}"

            value_pairs.append(value_pair)
        
        values_expr = ', '.join(value_pairs)
    else:
        values_expr = ', '.join([f"'{c}', {c}" for c in values])

    stack_select_expr = f"stack({len(values)}, {values_expr}) as ({variableColumnName}, {valueColumnName})"

    return df.selectExpr(*ids, stack_select_expr)

df_new = spark_unpivot_with_rename(
    df,
    ['Name'],
    ['Metric1', 'Metric2', 'Metric3'],
    renameMappings=[
        RenameMapping(original_name='Metric1', new_name='Metric10'),
        # RenameMapping(original_name='Metric2', new_name='Metric20'),
        RenameMapping(original_name='Metric3', new_name='Metric30')
        ]
)
# df = spark_unpivot(df, ['Name'], ['Metric1', 'Metric2', 'Metric3'])

df_new.show()

["'Metric10', Metric1", "'Metric2', Metric2", "'Metric30', Metric3"]
stack(3, 'Metric10', Metric1, 'Metric2', Metric2, 'Metric30', Metric3) as (variable, value)
+-------+--------+-----+
|   Name|variable|value|
+-------+--------+-----+
|  Alice|Metric10|   10|
|  Alice| Metric2|   40|
|  Alice|Metric30|   70|
|    Bob|Metric10|   20|
|    Bob| Metric2|   50|
|    Bob|Metric30|   80|
|Charlie|Metric10|   30|
|Charlie| Metric2|   60|
|Charlie|Metric30|   90|
+-------+--------+-----+

